# ADK Application Testing

This notebook demonstrates how to test an ADK (Agent Development Kit) application.
It covers both local and remote testing, both with Agent Engine and Cloud Run.

> **Note**: This notebook assumes that the agent files are stored in the `app` folder. If your agent files are located in a different directory, please update all relevant file paths and references accordingly.

<img src="https://github.com/GoogleCloudPlatform/agent-starter-pack/blob/main/docs/images/adk_logo.png?raw=true" width="400">



## Install dependencies

In [ ]:
!pip install google-cloud-aiplatform --upgrade
!pip install google-adk --upgrade


### Import libraries

In [ ]:
import json

import requests
import vertexai.agent_engines

## If you are using Agent Engine
See more documentation at [Agent Engine Overview](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview)

### Remote Testing

In [ ]:
# Replace with your Agent Engine ID
AGENT_ENGINE_ID = "projects/168027251845/locations/us-central1/reasoningEngines/32866601577480192"

In [ ]:
remote_agent_engine = vertexai.agent_engines.get(AGENT_ENGINE_ID)

In [ ]:
for event in remote_agent_engine.stream_query(message="whats my latest bill", user_id="test"):
    print(event)

In [ ]:
remote_agent_engine.register_feedback(
    feedback={
        "score": 5,
        "text": "Great response!",
        "invocation_id": "test-invocation-123",
        "user_id": "test",
    }
)

### Local Testing

You can import directly the AgentEngineApp class within your environment. 
To run the agent locally, follow these steps:
1. Make sure all required packages are installed in your environment
2. The recommended approach is to use the same virtual environment created by the 'uv' tool
3. You can set up this environment by running 'make install' from your agent's root directory
4. Then select this kernel (.venv folder in your project) in your Jupyter notebook to ensure all dependencies are available

In [1]:
# Uncomment the following lines if you're not using the virtual environment created by uv
import sys
import json
sys.path.append("../")

In [2]:
from app.agent import root_agent
from app.agent_engine_app import AgentEngineApp

agent_engine = AgentEngineApp(agent=root_agent)

In [3]:
for event in agent_engine.stream_query(message="Give me the cafe's near other starbucks", user_id="test"):
    print(json.dumps(event, indent=2))

C:\Users\reube\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\vertexai\preview\reasoning_engines\templates\adk.py:566: UserWarning: [EXPERIMENTAL] InMemoryCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  self._tmpl_attrs["credential_service"] = InMemoryCredentialService()
C:\Users\reube\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\google\adk\auth\credential_service\in_memory_credential_service.py:33: UserWarning: [EXPERIMENTAL] BaseCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__()


{
  "content": {
    "parts": [
      {
        "thought_signature": "CuQCAR_MhbaT0a36RsDm5_b82-pZd7U_uav1ziotQYRe36oiNeBrf1rC7WNRu8VcIL6odXdwRtclshwGeRZoGFvr_Cpb7lPsUhJgBbZqyurdGuevOExOM5klAfQIvaUVX18roKM0Ku4F8YsaPA8Fvdn-pWTXlBUGf-n1e1F6l1TGxyJrRt_uKVrSHNeCabKMVTTbU_1wiAStgXB6GjJdx_9Dy8AwlAaJsEsXTTEdKa6TohortNuX27UKqsbzsEtvKVXGT_Uj3dzcL0r9yFZdNdR-bk5SC1ghK4C-OcsMjfm5XI73-KkC_wvfTc_zpsDnBP3bHMT-oTlB1zQZhfNn2BLil-my6AORHYbUmXybn0lbiHDGIL9srNClBDGA0jAWT1YHracstnZh7KYvpGxdJICZofnZRe6R0G6gkfTc29VhqLsRLeRg69jIJPnZ39dLesT6ZbAmTqG8DsgbG7_JC5GGRW8y4-U=",
        "function_call": {
          "id": "adk-2e15c6cf-23d9-4389-8bff-e9c5fd4b1da5",
          "args": {
            "agent_name": "nearby_intent_agent"
          },
          "name": "transfer_to_agent"
        }
      }
    ],
    "role": "model"
  },
  "finish_reason": "STOP",
  "usage_metadata": {
    "candidates_token_count": 13,
    "candidates_tokens_details": [
      {
        "modality": "TEXT",
        "token_count": 13
      }
   

## If you are using Cloud Run

#### Remote Testing

For more information about authenticating HTTPS requests to Cloud Run services, see:
[Cloud Run Authentication Documentation](https://cloud.google.com/run/docs/triggering/https-request)

Remote testing involves using a deployed service URL instead of localhost.

Authentication is handled using GCP identity tokens instead of local credentials.

In [ ]:
ID_TOKEN = get_ipython().getoutput("gcloud auth print-identity-token -q")[0]

In [ ]:
SERVICE_URL = "YOUR_SERVICE_URL_HERE"  # Replace with your Cloud Run service URL

You'll need to first create a Session

In [ ]:
user_id = "test_user_123"
session_data = {"state": {"preferred_language": "English", "visit_count": 1}}

session_url = f"{SERVICE_URL}/apps/app/users/{user_id}/sessions"
headers = {"Content-Type": "application/json", "Authorization": f"Bearer {ID_TOKEN}"}

session_response = requests.post(session_url, headers=headers, json=session_data)
print(f"Session creation status code: {session_response.status_code}")
print(f"Session creation response: {session_response.json()}")
session_id = session_response.json()["id"]

Then you will be able to send a message

In [ ]:
message_data = {
    "app_name": "app",
    "user_id": user_id,
    "session_id": session_id,
    "new_message": {"role": "user", "parts": [{"text": "Hello! Weather in New york?"}]},
    "streaming": True,
}

message_url = f"{SERVICE_URL}/run_sse"
message_response = requests.post(
    message_url, headers=headers, json=message_data, stream=True
)

print(f"Message send status code: {message_response.status_code}")

# Print streamed response
for line in message_response.iter_lines():
    if line:
        line_str = line.decode("utf-8")
        if line_str.startswith("data: "):
            event_json = line_str[6:]
            event = json.loads(event_json)
            print(f"Received event: {event}")

### Local Testing

> You can run the application locally via the `make local-backend` command.

#### Create a session
 Create a new session with user preferences and state information


In [ ]:
user_id = "test_user_123"
session_data = {"state": {"preferred_language": "English", "visit_count": 1}}

session_url = f"http://127.0.0.1:8000/apps/app/users/{user_id}/sessions"
headers = {"Content-Type": "application/json"}

session_response = requests.post(session_url, headers=headers, json=session_data)
print(f"Session creation status code: {session_response.status_code}")
print(f"Session creation response: {session_response.json()}")
session_id = session_response.json()["id"]

#### Send a message
Send a message to the backend service and receive a streaming response


In [ ]:
message_data = {
    "app_name": "app",
    "user_id": user_id,
    "session_id": session_id,
    "new_message": {"role": "user", "parts": [{"text": "Hello! Weather in New york?"}]},
    "streaming": True,
}

message_url = "http://127.0.0.1:8000/run_sse"
message_response = requests.post(
    message_url, headers=headers, json=message_data, stream=True
)

print(f"Message send status code: {message_response.status_code}")

# Print streamed response
for line in message_response.iter_lines():
    if line:
        line_str = line.decode("utf-8")
        if line_str.startswith("data: "):
            event_json = line_str[6:]
            event = json.loads(event_json)
            print(f"Received event: {event}")